# 트랜스포머를 활용한 자연어처리

### 1장 트랜스포머 소개

In [2]:
from transformers import pipeline

classifier = pipeline('text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

    - 파이프라인이 자동으로 허깅페이스 허브에서 모델 가중치를 다운로드 함
    - 파이프라인 객체를 다시 만들 때는 가중치가 이미 다운로드되서, 캐싱된 버전을 사용한다는 메시지가 나옴
    - 기본적으로 text-classification 파이프라인은 감성 분석을 위해 설계된 모델을 사용하지만,
    다중 분류(multiclass classfication)과 다중 레이블 분류(multilabel classification)도 지원

In [18]:
import pandas as pd

text= '''Dera Amazon, last weeek I ordered on Optimus Prime action figure from your online store in Germany.
Unfortunately, when I opened the package, I deiscovered to my horror that I had been sent an action figure of Megatron insted!
As a lifelong enemy of trhe Decepticons, I hope you can understand my dilemma.'''

In [19]:
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.996499


    - 텍스트가 부정적이라고 확신함
    - 감성 분석 작업에서 파이프라인은 positive, negative 레이블 중 하나 반환

**`개체명 인식`**

    - 고객 피드백의 감성을 예측하는 일이 분석의 좋은 출발점이지만,
    피드백이 특정 제품과 서비스 중 무엇에 대한 건지 알고 싶을 때가 많음.
    - NLP에서는 제품, 장소, 사람 같은 실제 객체를 '개체명(named entity)'라고 하고,
    이러한 개체명을 텍스트에서 추출하는 작업을 '개체명 인식(Named Entity Recognition)'이라 함

In [20]:
ner_tagger = pipeline('ner', aggregation_strategy='simple')
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,LOC,0.377747,Der,0,3
1,ORG,0.432344,Amazon,5,11
2,MISC,0.985208,Optimus Prime,37,50
3,LOC,0.999717,Germany,91,98
4,PER,0.593265,Megatron,210,218
5,MISC,0.750991,Deceptico,255,264


    - 개체명을 감지하고 ORG(조직), LOC(위치) 같은 카테고리에 할당함
    - 모델 예측에 따라 단어를 그룹하기 하고자 한다면 AGGREGATION_STRATEGY 매개변수 사용
        => 예를 들어 개체명 'Optimus Prime'은 두 단어로 구성되지만 한아ㅢ 카테고리 MISC에 할당됨
    - 점수는 모델이 개체명을 얼마나 확신하는지 나타냄
    

**`질문 답변(Question answering)`**

    - 텍스트 구절과 함께 얻고 싶은 질문을 모델에 전달, 모델이 답변 텍스트 반환

In [21]:
reader = pipeline('question-answering')
question = 'what does the customer want?'
outputs = reader(question=question, context=text)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [22]:
pd.DataFrame([outputs])

,score,start,end,answer
0,0.420892,37,64,Optimus Prime action figure


    - 답이 위치한 문자 인덱스에 해당하는 start와 end 정수를 반환함
    - 답변을 텍스트에서 직접 추출하기 때문에 '추출적 질문 답변(extractive question answering)' 

**`요약(text summarization)`**

    - 긴 텍스트를 입력으로 받고, 사실이 모두 포함된 간단한 버전 생성
    - 모델이 논리적인 텍스트를 생성해야 하므로, 이전 문제보다 훨씬 복잡한 작업

In [23]:
summarizer = pipeline('summarization')
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [24]:
print(outputs[0]['summary_text'])

 Dera Amazon, last weeek I ordered on Optimus Prime action figure from your online store in Germany.Unfortunately, when I opened the package, I deiscovered to my horror that I had been sent an action figure of Megatron insted! As a lifelong enemy


    - 피드백이 이해할 수 없는 언어로 되어있다면, 구글 번역을 이용하거나
    직접 트랜스포머 모델을 사용하여 번역하는 방법이 있음

**`번역`**

- 요약과 같이 텍스트를 생성해 출력하는 작업

In [30]:
translator = pipeline('translation_en_to_de',
                     model='Helsinki-NLP/opus-mt-en-de')
outputs= translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Dera Amazon, letzte Woche bestellte ich auf Optimus Prime Action Figur von Ihrem Online-Shop in Deutschland. Leider, als ich das Paket öffnete, habe ich zu meinem Entsetzen entdeckt, dass ich eine Action Figur von Megatron insted geschickt worden war! Als lebenslanger Feind von Trhe Decepticons, hoffe ich, dass Sie mein Dilemma verstehen können,........................................................................................................................................................................................................................ 


    - 파이프라인의 기본 모델을 오버라이드(override)해서 애플리케이션에 가장 잘 맞는 모델 선택
    

**`텍스트 생성`**

In [31]:
generator = pipeline('text-generation')
response = 'Dear Bumblebee, I am sorry to hear that your order was mixed up.'
prompt = text + '\n\nCustomer service response:\n' + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dera Amazon, last weeek I ordered on Optimus Prime action figure from your online store in Germany.
Unfortunately, when I opened the package, I deiscovered to my horror that I had been sent an action figure of Megatron insted!
As a lifelong enemy of trhe Decepticons, I hope you can understand my dilemma.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I know that the orders are processed by the same company that sent the order, but this time the customer service response to my question simply stated that this was not my order. I ordered from a large shipping company. You will need to speak to a representative, because the company may have a problem.

Thanks for your time.


    - 답변의 아이디어 얻기

### `허깅페이스 데이터셋`

    - 메모리 매핑(memory mapping) 메커니즘을 활용해 램 부족을 피함
    파일 내용을 가상 메모리에 저장하고 여러 개의 프로세스로 더 효율ㅈ거으로 팡리을 수정함
    - 데이터 랭그링(data wrangling) 도구를 그대로 사용해도 좋음
    
### `허깅페이스 엑셀러레이트`

    - 사용자의 정의의 로직을 처리하는 일반적인 훈련 루프에 훈련 인프라에 필요한 추상화 층 추가